In [1]:
import requests
import time
import csv

#### Step 1: Checking the Location

In [2]:
def verifyStore(storeNum : int):
    """
    Grabs the most recent stores from https://www.wegmans.com/api/stores to verify if the queried store id exists.
    
    """
    url = "https://www.wegmans.com/api/stores"

    headers = {
    "Accept": "*/*",
    "Accept-Language": "en-US,en;q=0.9,es;q=0.8",
    "Cache-Control": "no-cache",
    "Pragma": "no-cache",
    "Referer": "https://www.wegmans.com/stores",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                "(KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36",
    }

    resp = requests.get(url, headers=headers)
    resp.raise_for_status() # sanity check 
    stores = resp.json()

    for store in stores:
        if store["storeNumber"] == storeNum:
            print(f"The queried store is {store['name']}.")
            return True
    
    print("ERR: This store number does not exist.")
    return False


#### Step 2: Grabbing all the categories available at the specific store id

In [3]:
def getCategories(storeId : int):
    """
    Get a list of all the subcategories that have data from a specific store id.

    """
    headers = {
        "Accept": "*/*",
        "Accept-Language": "en-US,en;q=0.9,es;q=0.8",
        "Cache-Control": "no-cache",
        "Pragma": "no-cache",
        "Referer": "https://www.wegmans.com/shop/categories",
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
            "(KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
        )
    }

    url = f"https://www.wegmans.com/api/categories/{storeId}"
    resp = requests.get(url, headers=headers)
    resp.raise_for_status() # sanity check 
    storeData = resp.json()
    categories = []
    for mainDepartment, subDepartments in storeData["subCategoriesWithData"].items():
        categories.extend([cat["key"] for cat in subDepartments])
    return categories

#### Step 4: For each of the subcategories, grab all the product data

In [4]:
def constructSession():
    """
    Builds a session for querying multiple times.
    
    """
    API_KEY = "9a10b1401634e9a6e55161c3a60c200d"
    APP_ID = "QGPPR19V8V"
    URL = f"https://{APP_ID}-dsn.algolia.net/1/indexes/*/queries"

    headers = {
        "x-algolia-api-key": API_KEY,
        "x-algolia-application-id": APP_ID,
        "user-agent": (
                    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) " 
                    "AppleWebKit/537.36 (KHTML, like Gecko) "
                    "Chrome/138.0.0.0 Safari/537.36"
        ),
        "content-type": "application/json",
        "origin": "https://www.wegmans.com",
        "referer": "https://www.wegmans.com/",
    }
    session = requests.Session()
    session.headers.update(headers)

    return session


In [309]:
def getAllBrands(subId : str, storeId : int, facetName = 'consumerBrandName',pageNum = 0, session = None):
    """
    Some subcategories have >1000 products. Further subdivision can be handled by grabbing consumer brand names.

    """
    
    if session == None:
        session = constructSession()

    APP_ID = "QGPPR19V8V"
    URL = f"https://{APP_ID}-dsn.algolia.net/1/indexes/*/queries"

    params = (
        "query=&"
        f"page={pageNum}&" 
        f"hitsPerPage={0}&"
        f"filters=storeNumber:{storeId} AND category.key:{subId} AND "
        "excludeFromWeb:false AND isSoldAtStore:true AND fulfilmentType:instore&"
        f"facets=%5B%22{facetName}%22%5D&"
        "maxValuesPerFacet=1000"
    )

    payload = {
        "requests": [
            {
                "indexName": "products",
                "params": params
            }
        ]
    }

    resp = session.post(URL, json=payload)
    resp.raise_for_status()
    res = resp.json()["results"][0]
    return res['facets']['consumerBrandName']


In [ ]:
def getAllBrandsNums(subId: str, storeId: int, facetName: str = 'consumerBrandName',
                 pageNum: int = 0, session=None) -> int:
    """
    Returns the count of unique consumerBrandName values for the given store+subcategory.
    """
    if session is None:
        session = constructSession()

    APP_ID = "QGPPR19V8V"
    URL    = f"https://{APP_ID}-dsn.algolia.net/1/indexes/*/queries"

    params = (
        "query=&"
        f"page={pageNum}&"
        "hitsPerPage=0&"
        f"filters=storeNumber:{storeId} AND category.key:{subId} AND "
        "excludeFromWeb:false AND isSoldAtStore:true AND fulfilmentType:instore&"
        f"facets=%5B%22{facetName}%22%5D&"
        "maxValuesPerFacet=1000"
    )

    payload = {
        "requests": [
            {
                "indexName": "products",
                "params": params
            }
        ]
    }

    resp = session.post(URL, json=payload)
    resp.raise_for_status()
    res = resp.json()["results"][0]

    # get the dict of { brandName: count }
    brands_dict = res.get("facets", {}).get(facetName, {})
    # return just the number of distinct brands
    return len(brands_dict)


In [306]:
def getSubCategoryData(subId : str, storeId : int, hits = 1000, pageNum = 0, brandName = None, session = None):
    """
    Given a store id and a subcategory id, get the data in the form of a dictionary.

    """
    
    if session == None:
        session = constructSession()

    APP_ID = "QGPPR19V8V"
    URL = f"https://{APP_ID}-dsn.algolia.net/1/indexes/*/queries"

    params = (
        "query=&"
        f"page={pageNum}&" # hitsPerPage should cover everything if the subcategories are the ones being explored
        f"hitsPerPage={hits}&"
        f"filters=storeNumber:{storeId} AND category.key:{subId} AND "
        "excludeFromWeb:false AND isSoldAtStore:true AND fulfilmentType:instore"
    )

    if brandName:
        payload = {
            "requests": [
                {
                    "indexName": "products",
                    "params": params,
                    "facetFilters": [[f"consumerBrandName:{brandName}"]]
                }
            ]
        }      
    else:
        payload = {
            "requests": [
                {
                    "indexName": "products",
                    "params": params
                }
            ]
        }


    resp = session.post(URL, json=payload)
    resp.raise_for_status()
    res = resp.json()["results"][0]

    allHits = []

    # edge case where a subcategory nbHits > Hits
    if res['nbHits'] > len(res['hits']):
        print(subId)
        allBrands = getAllBrands(subId, storeId, session = session)
        # print(getAllBrandsNums(subId, storeId, session = session))
        for brandName in allBrands:
            allHits.extend(getSubCategoryData(subId, storeId, brandName = brandName, session = session))
            time.sleep(0.2)
        
    else:
        allHits = res['hits']

    # print("all:" + str(len(allHits)) +  " nbHits: " + str(res['nbHits']))
    return allHits


In [297]:
def getSubCategoryDataIterative(subId, storeId, hits=300, session=None):
    session = session or constructSession()
    APP_ID = "QGPPR19V8V"
    URL    = f"https://{APP_ID}-dsn.algolia.net/1/indexes/*/queries"

    def fetch_page(page):
        params = (
            "query=&"
            f"page={page}&"
            f"hitsPerPage={hits}&"
            "filters="
            f"storeNumber:{storeId} AND "
            f"category.key:{subId} AND "
            "excludeFromWeb:false AND "
            "isSoldAtStore:true AND "
            "fulfilmentType:instore"
        )
        payload = {"requests":[{"indexName":"products","params":params}]}
        # add timeout and retry if you want here
        r = session.post(URL, json=payload, timeout=(5,30))
        r.raise_for_status()
        return r.json()["results"][0]

    # first page
    first = fetch_page(0)
    all_hits = first["hits"]
    total_pages = first.get("nbPages", 1)

    # remaining pages
    for p in range(1, total_pages):
        print(f"Fetching page {p}/{total_pages-1}")
        page_data = fetch_page(p)
        all_hits.extend(page_data["hits"])

    return all_hits


In [312]:
def filteredQuery(subId : str, storeId : int, facet_filter, hits = 1000, session = None):
    
    if session == None:
        session = constructSession()

    APP_ID = "QGPPR19V8V"
    URL = f"https://{APP_ID}-dsn.algolia.net/1/indexes/*/queries"

    params = (
        "query=&"
        f"page={0}&" # hitsPerPage should cover everything if the subcategories are the ones being explored
        f"hitsPerPage={hits}&"
        f"filters=storeNumber:{storeId} AND category.key:{subId} AND "
        "excludeFromWeb:false AND isSol"
        "dAtStore:true AND fulfilmentType:instore"
    )

    payload = {
        "requests": [
            {
                "indexName": "products",
                "params": params,
                "facetFilters": [facet_filter]
            }
        ]
    }      
    
    resp = session.post(URL, json=payload)
    resp.raise_for_status()
    res = resp.json()["results"][0]

    return res['hits']

In [329]:
def getSubCategoryDataBlobs(subId : str, storeId : int, hits = 1000, brandName = None, session = None):
    """
    Given a store id and a subcategory id, get the data in the form of a dictionary.

    """
    
    if session == None:
        session = constructSession()

    APP_ID = "QGPPR19V8V"
    URL = f"https://{APP_ID}-dsn.algolia.net/1/indexes/*/queries"

    params = (
        "query=&"
        f"page={0}&" # hitsPerPage should cover everything if the subcategories are the ones being explored
        f"hitsPerPage={hits}&"
        f"filters=storeNumber:{storeId} AND category.key:{subId} AND "
        "excludeFromWeb:false AND isSoldAtStore:true AND fulfilmentType:instore"
    )

    if brandName:
        payload = {
            "requests": [
                {
                    "indexName": "products",
                    "params": params,
                    "facetFilters": [[f"consumerBrandName:{brandName}"]]
                }
            ]
        }      
    else:
        payload = {
            "requests": [
                {
                    "indexName": "products",
                    "params": params
                }
            ]
        }


    resp = session.post(URL, json=payload)
    resp.raise_for_status()
    res = resp.json()["results"][0]

    allHits = []

    # edge case where a subcategory nbHits > Hits
    if res['nbHits'] > len(res['hits']):
        brand_dct = getAllBrands(subId, storeId)

        curr_filter = []
        curr_sum = 0
        for brand, total in brand_dct.items():
            if curr_sum + total >= 1000:
                curr_sum = total

                # do the query, add to allHits
                allHits.extend(filteredQuery(subId, storeId, curr_filter, session = session))

                curr_filter = [f'consumerBrandName:{brand}']
            else:
                curr_filter.append(f'consumerBrandName:{brand}')
                curr_sum += total
        
        if curr_filter: # still some categories left
            allHits.extend(filteredQuery(subId, storeId, curr_filter, session = session))
        
    else:
        allHits = res['hits']
    


    # print("all:" + str(len(allHits)) +  " nbHits: " + str(res['nbHits']))
    return allHits



In [84]:
bruh = getAllBrandsNums('2957337', 156).values()

In [322]:
test = getSubCategoryDataBlobs('5108174', 156, hits=1000)

In [323]:
len(test)

1136

In [ ]:
brand_dct = getAllBrands('5108174', 156)

curr_filter = []
curr_sum = 0
for brand, total in brand_dct:
    if curr_sum + total >= 1000:
        curr_sum = total

        # do the query, add to allHits

        curr_filter = [f'consumerBrandName:{brand}']
    else:
        curr_filter.append(f'consumerBrandName:{brand}')
        curr_sum += total

In [308]:
len(test)

1136

In [298]:
new_test = getSubCategoryDataIterative('5108174', 156)

Fetching page 1/3
Fetching page 2/3
Fetching page 3/3


In [299]:
len(new_test)

1000

In [281]:
len(new_test)

1000

In [274]:
len(test['hits'])

999

In [275]:
test['nbPages']

2

In [113]:
lst = getAllBrands('2957337', 156)

In [114]:
filter_lst = []
for brand in lst:
    filter_lst.append(f"NOT consumerBrandName:{brand}")

In [116]:
" AND ".join(filter_lst)

"NOT consumerBrandName:Wegmans AND NOT consumerBrandName:Poppi AND NOT consumerBrandName:Starbucks AND NOT consumerBrandName:Gatorade AND NOT consumerBrandName:Traditional Medicinals AND NOT consumerBrandName:Bigelow AND NOT consumerBrandName:Polar AND NOT consumerBrandName:Celestial Seasonings AND NOT consumerBrandName:Pepsi AND NOT consumerBrandName:La Colombe AND NOT consumerBrandName:Red Bull AND NOT consumerBrandName:Peet's Coffee AND NOT consumerBrandName:Ocean Spray AND NOT consumerBrandName:Fever-Tree AND NOT consumerBrandName:Coca-Cola AND NOT consumerBrandName:Alani Nu AND NOT consumerBrandName:Canada Dry AND NOT consumerBrandName:Core Power AND NOT consumerBrandName:Dunkin' AND NOT consumerBrandName:Ghost AND NOT consumerBrandName:Monster AND NOT consumerBrandName:Yogi AND NOT consumerBrandName:Dr Pepper AND NOT consumerBrandName:Ito En AND NOT consumerBrandName:MiO AND NOT consumerBrandName:Stur AND NOT consumerBrandName:Counter Culture AND NOT consumerBrandName:Snapple AND

In [371]:
def fetch_unbranded(subId, storeId, **opts):
    """
    Fetches only products with NO consumerBrandName set.
    """
    session = opts.pop("session", None) or constructSession()

    APP_ID = "QGPPR19V8V"
    URL = f"https://{APP_ID}-dsn.algolia.net/1/indexes/*/queries"
    lst = getAllBrands(subId, storeId)
    filter_lst = []
    for brand in lst:
        filter_lst.append(f"NOT consumerBrandName:%27{brand}%27")
    negative_filter = " AND ".join(filter_lst[0:3])
    print(negative_filter)
    params = (
        "query=&"
        f"page={opts.get('pageNum',0)}&"
        f"hitsPerPage={opts.get('hits',1000)}&"
        f"filters=storeNumber:{storeId} AND category.key:{subId} AND "
        "excludeFromWeb:false AND isSoldAtStore:true AND fulfilmentType:instore AND "
        f"{negative_filter}"
    )
    payload = {"requests":[{"indexName":"products","params":params}]}
    r = session.post(URL, json=payload); r.raise_for_status()
    return r.json()["results"][0]



checkpoint: manual check to see what is missing versus what is in my dataset

In [372]:
A = fetch_unbranded('5108174', 1, hits=1000, pageNum=0)
print("Unbranded nbHits =", A["nbHits"])


NOT consumerBrandName:%27Wegmans%27 AND NOT consumerBrandName:%27Pepperidge Farm%27 AND NOT consumerBrandName:%27Goldfish%27
Unbranded nbHits = 1571


In [ ]:
B = fetch_unbranded('5108174', 156, hits=1000, pageNum=0)
print("Unbranded nbHits =", B["nbHits"])

NOT consumerBrandName:'Wegmans' AND NOT consumerBrandName:'Snyder's of Hanover'


HTTPError: 400 Client Error: Bad Request for url: https://qgppr19v8v-dsn.algolia.net/1/indexes/*/queries

In [44]:
STORE_ID = 156
session = constructSession()
allCategories = getCategories(STORE_ID)

Step 5: main function

In [45]:
fieldnames = [
    "productName",
    "upc",
    "lastUpdated",
    "productKeywords",
    "isAvailable",
    "consumerBrandName",
    "popularTags",
    "filterTags",
    "webProductDescription",
    "price_inStore_unitPrice",
    "price_inStore_amount",
    "skuId",
    "taxCode",
    "packSize",
    "productId",
    "productDescription"
]



In [330]:
STORE_ID = 156
session = constructSession()
allCategories = getCategories(STORE_ID)


In [331]:
numProds = 0
with open(f"wegmans_store_{STORE_ID}_{int(time.time())}.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames = fieldnames, lineterminator="\n")
    writer.writeheader()

    for subcategory in allCategories:
        subData = getSubCategoryDataBlobs(subcategory, STORE_ID, session = session)
        for product in subData:
            numProds += 1
            row = {}
            for attr in fieldnames:
                if attr.startswith("price_inStore_"):
                    subkey = attr.split("_")[-1]  
                    row[attr] = product.get("price_inStore", {}).get(subkey, "")
                else:
                    row[attr] = product.get(attr, "")
            writer.writerow(row)
        time.sleep(0.2)

In [ ]:
getSubCategoryData()

In [48]:
print(numProds)

14799


In [160]:
import pandas as pd

In [332]:
df = pd.read_csv('wegmans_store_156_1752988156.csv')

In [333]:
df.shape

(14798, 16)

In [252]:
verify = set(df['productId'])

In [253]:
A['hits'][0]['productId']

'543949'

In [254]:
for prod in A['hits']:
    if int(prod['productId']) not in verify:
        print(prod)
        culp = prod

for prod in B['hits']:
    if int(prod['productId']) not in verify:
        print(prod)
        culp = prod
    else:
        ex = prod

{'loyaltyInstoreDiscount': [], 'categories': {'lvl0': 'Grocery', 'lvl1': 'Grocery > Chips & Snack Foods', 'lvl2': 'Grocery > Chips & Snack Foods > Chips'}, 'loyaltyDeliveryDiscount': [], 'maxQuantity': 99, 'taxCode': '0568-000', 'productDescription': 'Lays Salt & Vinegar Potato Chips Party Size 12.5 OZ', 'hasOffers': True, 'requiredMinimumAgeToBuy': 0, 'lastUpdated': '2025-07-16T09:09:03+00:00', 'images': ['https://images.wegmans.com/is/image/wegmanscsprod/804790_PrimaryImage?eTag=1746539151180'], 'webProductDescription': 'Potato Chips, Salt & Vinegar', 'productID': '804790', 'planogram': {'aisle': '05A', 'shelf': '1', 'aisleSide': 'L', 'section': '2'}, 'fulfilmentType': ['instore', 'pickup', 'delivery'], 'bottleDeposit': 0, 'discountType': '', 'isSoldAtStore': True, 'filterTags': ['Family Pack'], 'upc': ['00028400310581'], 'slug': '804790-Potato-Chips-Salt-Vinegar', 'categoryPageId': ['Grocery', 'Grocery > Chips & Snack Foods', 'Grocery > Chips & Snack Foods > Chips'], 'isAlcoholItem'

In [258]:
culp

{'loyaltyInstoreDiscount': [],
 'categories': {'lvl0': 'Grocery',
  'lvl1': 'Grocery > Chips & Snack Foods',
  'lvl2': 'Grocery > Chips & Snack Foods > Chips'},
 'loyaltyDeliveryDiscount': [],
 'maxQuantity': 99,
 'taxCode': '0568-000',
 'productDescription': 'Lays Salt & Vinegar Potato Chips Party Size 12.5 OZ',
 'hasOffers': True,
 'requiredMinimumAgeToBuy': 0,
 'lastUpdated': '2025-07-16T09:09:03+00:00',
 'images': ['https://images.wegmans.com/is/image/wegmanscsprod/804790_PrimaryImage?eTag=1746539151180'],
 'webProductDescription': 'Potato Chips, Salt & Vinegar',
 'productID': '804790',
 'planogram': {'aisle': '05A', 'shelf': '1', 'aisleSide': 'L', 'section': '2'},
 'fulfilmentType': ['instore', 'pickup', 'delivery'],
 'bottleDeposit': 0,
 'discountType': '',
 'isSoldAtStore': True,
 'filterTags': ['Family Pack'],
 'upc': ['00028400310581'],
 'slug': '804790-Potato-Chips-Salt-Vinegar',
 'categoryPageId': ['Grocery',
  'Grocery > Chips & Snack Foods',
  'Grocery > Chips & Snack Food

In [257]:
culp['consumerBrandName']

KeyError: 'consumerBrandName'

In [204]:
len(A) + len(B)

32

In [205]:
culp

{'productName': 'Seltzer Water, Blackberry',
 'slug': '841182-Seltzer-Water-Blackberry',
 'bottleDeposit': 0.05,
 'requiredMinimumAgeToBuy': 0,
 'productDescription': "Hal's New York Blackberry Sparkling Water - Single 20 FO",
 'excludeFromWeb': False,
 'loyaltyDeliveryDiscount': [],
 'maxQuantity': 99,
 'categoryNodes': {'lvl0': 'Grocery',
  'lvl1': 'Beverages',
  'lvl2': 'Sparkling & Seltzer Water'},
 'lastUpdated': '2025-07-14T23:33:01+00:00',
 'category': [{'name': 'Sparkling & Seltzer Water',
   'key': '5681772',
   'seo': 'Sparkling-Seltzer-Water-5681772'},
  {'name': 'Beverages', 'key': '2957337', 'seo': 'Beverages-2957337'},
  {'name': 'Grocery', 'key': '2957334', 'seo': 'Grocery-2957334'}],
 'isSoldByWeight': False,
 'averageStarRating': 0,
 'isNewItem': False,
 'isAvailable': True,
 'onlineApproxUnitWeight': 0,
 'upc': ['00855036005266', '10855036005263'],
 'popularTags': [],
 'productKeywords': ['blackberry sparkling water', 'water'],
 'productID': '841182',
 'price_delivery

In [193]:
verifyStore(156)

The queried store is Astor Place.


True

In [188]:
prod.keys()

dict_keys(['isIWSProduct', 'categoryFacets', 'productKeywords', 'consumerBrandName', 'popularTags', 'price_deliveryLoyalty', 'nutrition', 'keywords', 'isNewItem', 'wellnessKeys', 'webProductDescription', 'storeNumber', 'loyaltyInstoreDiscount', 'skuId', 'images', '_highlightResult', 'productName', 'productDescription', 'onlineSellByUnit', 'averageStarRating', 'reviewCount', 'onlineApproxUnitWeight', 'upc', 'packSize', 'categories', 'isSoldByWeight', 'isLoyalty', 'bottleDeposit', 'fulfilmentType', 'taxCode', 'ingredients', 'restrictedOTC', 'categoryNodes', 'category', 'hasOffers', 'price_inStoreLoyalty', 'price_delivery', 'digitalCouponsOfferIds', 'requiredMinimumAgeToBuy', 'lastUpdated', 'productId', 'discountType', 'categoryPageId', 'planogram', 'isAlcoholItem', 'productID', 'price_inStore', 'filterTags', 'objectID', 'excludeFromWeb', 'consumerSubBrandName', 'slug', 'ebtEligible', 'isSoldAtStore', 'isAvailable', 'loyaltyDeliveryDiscount', 'maxQuantity'])

In [17]:
numProds/2

14799.0

#### Step 6: safety checks

In [69]:
def getNumData(storeId : int, hits = 1000, pageNum = 0, brandName = None, session = None):
    """
    Given a store id and a subcategory id, get the data in the form of a dictionary.

    """
    
    if session == None:
        session = constructSession()

    APP_ID = "QGPPR19V8V"
    URL = f"https://{APP_ID}-dsn.algolia.net/1/indexes/*/queries"

    params = (
        "query=&"
        f"page={pageNum}&" # hitsPerPage should cover everything if the subcategories are the ones being explored
        f"hitsPerPage={hits}&"
        f"filters=storeNumber:{storeId} AND "
        "excludeFromWeb:false AND isSoldAtStore:true AND fulfilmentType:instore"
    )

    if brandName:
        payload = {
            "requests": [
                {
                    "indexName": "products",
                    "params": params,
                    "facetFilters": [[f"consumerBrandName:{brandName}"]]
                }
            ]
        }      
    else:
        payload = {
            "requests": [
                {
                    "indexName": "products",
                    "params": params
                }
            ]
        }


    resp = session.post(URL, json=payload)
    resp.raise_for_status()
    res = resp.json()["results"][0]

    return res


In [335]:
getNumData(1)['nbHits']

46806

In [338]:
getNumData(1)['nbHits']

46806

In [70]:
sanity = getNumData(156)

In [72]:
sanity.keys()

dict_keys(['hits', 'nbHits', 'hitsPerPage', 'index', 'page', 'nbPages', 'processingTimeMS', 'processingTimingsMS', 'exhaustiveNbHits', 'exhaustiveTypo', 'query', 'queryAfterRemoval', 'params', 'renderingContent', 'exhaustive', 'aiReRanking'])

In [74]:
sanity['exhaustive']

{'nbHits': True, 'typo': True}

In [373]:
html_ = """
<ul class="list tw:px-1 tw:pl-3 tw:py-4"><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Wegmans (310)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Pepperidge Farm (54)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Goldfish (43)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Lay's (41)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Snyder's of Hanover (41)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Oreo (38)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Tostitos (34)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Utz (34)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Ritz (33)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Cheez-It (28)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Doritos (26)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Triscuit (26)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Cape Cod (25)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Little Debbie (23)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Chips Ahoy! (22)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Simple Mills (22)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Siete (20)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Keebler (19)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Voortman Bakery (17)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Pringles (16)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Schar (16)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Snack Factory (16)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Frito Lay (15)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Tate's Bake Shop (15)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Annie's (14)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Cheetos (14)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Hostess (14)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Wise (14)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>belVita (14)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Kernel Season's (13)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Lance (13)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Jack Link's (12)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Ruffles (12)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Welch's (11)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Planters (10)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Quaker (10)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>The Good Crisp Company (10)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Wheat Thins (10)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Archway (9)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Crunchmaster (9)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Fritos (9)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>LesserEvil (9)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Lundberg Family Farms (9)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Orville Redenbacher's (9)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Partake (9)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Smartfood (9)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Stacy's (9)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Country Archer (8)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Entenmann's (8)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Goodie Girl (8)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>MadeGood (8)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>PopCorners (8)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Premium (8)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Real Food From the Ground Up (8)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Rold Gold (8)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Xochitl (8)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Back to Nature (7)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Club (7)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Pace (7)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Setton Farms (7)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>SkinnyPop (7)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Stauffer's (7)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Sun-Maid (7)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Angie's Boomchickapop (6)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Chomps (6)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Dot's Homestyle Pretzels (6)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Good Thins (6)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Honey Maid (6)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Kettle Brand (6)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Mary Macleod's Shortbread (6)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Mauna Loa (6)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Milton's (6)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Newman's Own (6)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Ocean Spray (6)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Rustic Bakery (6)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Stella D'oro (6)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>SunChips (6)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>THE DAILY CRAVE (6)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Toasteds (6)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Town House (6)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Boulder Canyon (5)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Chester's (5)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Chex Mix (5)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Drizzilicious (5)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>From The Ground Up (5)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Harvest Snaps (5)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Herdez (5)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Jolly Time (5)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Nabisco (5)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Nutty &amp; Fruity (5)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Pamela's (5)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Quinn (5)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Sunsweet (5)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Teddy Grahams (5)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Terra (5)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Top Seedz (5)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Unique Snacks (5)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Wilde (5)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Wonderful Pistachios (5)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Absolutely! Gluten Free (4)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Bare (4)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Biena (4)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Blue Diamond Almonds (4)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Brothers All Natural (4)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Carr's (4)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Dewey's Bakery (4)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Emmy's Organics (4)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Every Body Eat (4)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Fit Joy (4)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Glutino (4)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Good Health (4)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Gratify (4)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Hippeas (4)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Jennies (4)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Kellogg's (4)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Late July (4)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Maine Crisp (4)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Natural Delights (4)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Nilla (4)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Nutter Butter (4)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Parm Crisps (4)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Pop-Secret (4)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Simply 7 (4)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>SkinnyDipped (4)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Tastykake (4)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Wasa (4)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Bachman (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Barnum's Animals (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Bella Lucia (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Blue Diamond (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Breton (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Catalina Crunch (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Chi-Chi's (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Country Archer Provisions (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Craize (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Cybele's (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Drybox (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Edward &amp; Sons (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Firehook (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Funyuns (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Go Raw (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Goya (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Happy Snacks (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Healing Home Foods (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Hubs (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Khloud (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Kinnikinnick (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Mateo's (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Moon Cheese (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Mother's (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Newtons (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Nonni's (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Pirate's Booty (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>RW Garcia (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Real Foods (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Realsy Foods, Inc. (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Reese's (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Rule Breaker (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Santitas (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Sensible Portions (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Solely (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Spudsy (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Steve &amp; Andy's (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Sweet Loren's (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>The GFB (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>The Greater Knead (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Tochi (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Tutti (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Vermont Smoke &amp; Cure (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Wonderful (3)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>4505 Meats (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Artisan Tropic (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Bear (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Biscoff (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Black Jewell (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Brami (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Brownie Brittle (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Cabot (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Combos (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>David (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Drake's (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Easy Cheese (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Element (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Ella's Flats (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Famous Amos (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Farmer Jon's (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Flock (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Flora (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Food Should Taste Good (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Freakin' Wholesome® (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Free To Eat (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Funables (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Garden of Eatin' (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Golden Oreo (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Gushers (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Hippie Snacks (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Irvins (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Jackson's (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Like Air (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Loacker (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Made Good (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Made in Nature (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Mariani (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Maxine's Heavenly (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Mrs. Renfro's (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Munchies (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>New York Chips (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Old London (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>P-Nuff Crunch (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Patagonia (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Pepperidge Farm® (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Pipcorn (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Pirates Booty (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Popchips (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Poppy (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Prana Organic (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Pretzelized Snacks (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Primo &amp; Marys (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Purplesful Snacking (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Quest (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Rib Rack (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Rivalz (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>SeedWise (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Seneca (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Sheila G's (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Slim Jim (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Snack Mates (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Snyder's-Lance (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Stellar (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Stonehedge Farms (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Stony Brook (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Sun Tropics (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Taali (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Terrell's (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Tillamook Country Smoker (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Todd's Better Snacks (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>VERMONT (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Veggie Rob's (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Walker's (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Wege of Hanover (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Westminster Bakers Co. (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Wow Baking Company (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Zesty Z (2)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>34 Degrees (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>5th Season (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Andy Capp's (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Anthem Snacks (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Arrowhead Mills (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Bada Bean Bada Boom (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Betty Crocker (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Biscos (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Black Forest (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Boulder (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Brothers (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Bugles (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Cabaret (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Candy Pop (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Chessmen (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Chicken in a Biskit (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Cracker Jack (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Craigs Creamery (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Cybeles (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Daily Crave (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Daily Crunch (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Dare (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>DeLallo (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Earth Balance (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Eat Smart (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Epic (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Evil Snacks (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Fruit Roll-Ups (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Fruit by the Foot (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>G.H. Cretors (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Galil (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>General Mills (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Ginger Snaps (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Grains First (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Guinness (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Hawaiian Host (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Hershey's (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Iwon (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Iwon Organics (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Jax (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Jiffy Pop (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Kettle (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Kim's (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Kinder (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>LE PAIN DES FLEURS (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>La Dolce Vita (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>La Preferida (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Lays (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Live Love Pop (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Lorna Doone (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Louisville Vegan Jerky (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Louisville Vegan Jerky Co (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Mary's Gone Crackers (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Matilde Vicenzi (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Mi Nina (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Mi-Del (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Mott's (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Munchos (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Nut-rition (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Nutella B-Ready (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Nutella Biscuits (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Orville Redenbachers (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Patagonia Provisions (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Pinwheels (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Qwackers (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Royal Dansk (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>SARATOGA CHIPS (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Saratoga (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Savor by Suzie (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Seapoint Farms (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Sensational Brands (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Sesmark (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Simply (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Snacklins (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Snake River Farms (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Sociables (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Tate's (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Tillamook (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Toasted (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Tortilleria Mi Nina (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Vacadillos (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Vegan Rob's (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Vinta (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Wilde Chips (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Woodstock (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Wow (1)</label></li><li><label class="component--shop-sidebar-filters-checkbox global--form-text"><input class="checkbox" type="checkbox"><span aria-hidden="true" class="checkbox-icon"><svg class="" data-testid="IconCheckboxUnselected" fill="none" height="20" viewBox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg"><rect fill="white" height="19" rx="3.5" stroke="#6F6F6F" width="19" x="0.5" y="0.5"></rect></svg></span>Zapp's (1)</label></li></ul>
"""

In [375]:
from bs4 import BeautifulSoup

# 2. Construct a BeautifulSoup object
soup = BeautifulSoup(html_, "html.parser")

# 3. Find the <ul> (or all of them)
ul = soup.find("ul")         # or soup.find_all("ul") for multiple lists

# 4. Pull out each <li>’s text
items = [li.get_text(strip=True) for li in ul.find_all("li")]

print(items)
# → ['First item', 'Second item with bold text', 'Third item']


['Wegmans (310)', 'Pepperidge Farm (54)', 'Goldfish (43)', "Lay's (41)", "Snyder's of Hanover (41)", 'Oreo (38)', 'Tostitos (34)', 'Utz (34)', 'Ritz (33)', 'Cheez-It (28)', 'Doritos (26)', 'Triscuit (26)', 'Cape Cod (25)', 'Little Debbie (23)', 'Chips Ahoy! (22)', 'Simple Mills (22)', 'Siete (20)', 'Keebler (19)', 'Voortman Bakery (17)', 'Pringles (16)', 'Schar (16)', 'Snack Factory (16)', 'Frito Lay (15)', "Tate's Bake Shop (15)", "Annie's (14)", 'Cheetos (14)', 'Hostess (14)', 'Wise (14)', 'belVita (14)', "Kernel Season's (13)", 'Lance (13)', "Jack Link's (12)", 'Ruffles (12)', "Welch's (11)", 'Planters (10)', 'Quaker (10)', 'The Good Crisp Company (10)', 'Wheat Thins (10)', 'Archway (9)', 'Crunchmaster (9)', 'Fritos (9)', 'LesserEvil (9)', 'Lundberg Family Farms (9)', "Orville Redenbacher's (9)", 'Partake (9)', 'Smartfood (9)', "Stacy's (9)", 'Country Archer (8)', "Entenmann's (8)", 'Goodie Girl (8)', 'MadeGood (8)', 'PopCorners (8)', 'Premium (8)', 'Real Food From the Ground Up (8)

In [383]:
sum([int(i.split(" ")[-1].strip("()")) for i in items])

1977

In [385]:
final = getSubCategoryDataBlobs('5108174', 1)

In [387]:
len(final)

1977

some notes:
- I validated that the sum of all the brands matches w/ the website
- Lays salt & vinegar potato chips party size 12.5 oz